In [1]:
import requests
import pandas as pd
import datetime
from barcelona_keys import key as weather_key


def getting_daily_weather(year,month,day,variables_selected,codiEstacio='D5'):
    
    url=f'https://api.meteo.cat/xema/v1/estacions/mesurades/{codiEstacio}/{year}/{month}/{day}'
    response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key}).json()
    list_variables_codes=[variable['code'] for variable in variables_selected]
    list_variables_names=[variable['name'] for variable in variables_selected]
    df=pd.DataFrame()
    for variable in response[0]['variables']:
        if variable['codi'] in list_variables_codes:
            #print(variable)
            data=pd.DataFrame.from_records(variable['lectures'])[['data','valor']]
            data.rename(columns={'valor':list_variables_names[list_variables_codes.index(variable['codi'])]},inplace=True)

            if df.empty:
                df=data.copy()
            else:
                df=df.merge(data, on='data')
        
    return df


def getting_yearly_weather(year):
    start_date = datetime.date(year, 1, 1)
    end_date = datetime.date(year, 12, 31)
 
    # delta time
    delta = datetime.timedelta(days=1)
    yearly_weather=pd.DataFrame()
    while (start_date <= end_date):
        #print(start_date, end="\n")
        daily_weather=getting_daily_weather(year,'{:02d}'.format(start_date.month),'{:02d}'.format(start_date.day),variables_selected,codiEstacio='D5')
        yearly_weather=pd.concat([yearly_weather,daily_weather])

        start_date += delta
    yearly_weather.to_csv('./data/weatherbarcelona'+str(year)+'.csv',index=False)
    print("DONE",year)
    #return yearly_weather.reset_index(inplace=True)
    
    
##Figuring out estacions

url='https://api.meteo.cat/xema/v1/estacions/metadades?estat=ope&data=2023-06-01Z'
response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key})
estacions=response.json()
#estacions from Barcelona
estacions_barcelona=[{'code': estacio['codi'],'name':estacio['nom']} for estacio in estacions if estacio['municipi']['nom']=='Barcelona']

#figuring out variables
url='https://api.meteo.cat/xema/v1/variables/mesurades/metadades'
variables = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key}).json()
selected_variables=[26,30,32,33,34,35,36,38]#selection done by going over all variables
variables_selected=[{'code':variable['codi'],'name':variable['nom'],'units':variable['unitat']} for variable in variables if variable['codi'] in selected_variables]


#single year
ANY=input("What year do you want? ")
getting_yearly_weather(str(ANY))
#doing each year for the fist time
first_year=2011
last_year=2022
for year in range(first_year,last_year+1):
    getting_yearly_weather(year)

In [11]:
#doing each year for the fist time
first_year=2011
last_year=2022
for year in range(first_year,last_year+1):
    getting_yearly_weather(year)

[]

In [72]:
#Possible stations: D5,X2,X4,X8

In [127]:
##Doing it by hand first time
query="SELECT  data_lectura,codi_variable, valor_lectura \
where codi_estacio='D5' and codi_variable in ('30','32','33','34','35','36') \
and data_lectura >= '2010-01-01' and data_lectura <'2023-01-01' and codi_base='SH'"
url=f'https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json?$query={query}'
response=requests.get(url)
exemple=response.json()#doing each year for the fist time
##pending some paging--I can only get 1000 per shot

200

In [96]:
##Downloading it directly from open data gencat 
import requests
import pandas as pd
import datetime
from barcelona_keys import key as weather_key

##Prepping final weather file

weather=pd.read_csv('/Users/fcbnyc/Downloads/weatherBarcelona2010_2022.csv')
selected_columns=['DATA_LECTURA','CODI_VARIABLE','VALOR_LECTURA']
final_weather=weather[selected_columns].copy()
final_weather.columns=['date','variable','value']
final_weather['date']=pd.to_datetime(final_weather.date)
final_weather.sort_values('date',inplace=True)

##Prepping datetime df

hourly_range=pd.date_range(start='2010-01-01',end='2023-01-1',freq='H')
df_hourly=pd.DataFrame(index=hourly_range)

halfhourly_range=hourly_range+pd.Timedelta('30min')

df_halfhourly=pd.DataFrame(index=halfhourly_range[:-1])

df_datetime=pd.concat([df_hourly,df_halfhourly]).sort_index()[:-1]
df=df_datetime.copy()
variable_dict={'30':'windspeed10m','32':'temp','33': 'humidity',\
                '34':'pressure','35':'precipitation','36': 'solar_irridation'}

for variable in final_weather.variable.unique():
    new_df=final_weather[final_weather.variable==variable][['date','value']].set_index('date')
    new_df.rename(columns={'value':variable_dict[str(variable)]},inplace=True)
    df=pd.merge(df,new_df,how='left',left_index=True,right_index=True)
    #print(df.shape)
df.interpolate(inplace=True)
df.to_csv('../data/weatherBCN2010_2022.csv',index=False)
df.tail()

,solar_irridation,temp,precipitation,pressure,windspeed10m,humidity
2022-12-31 21:30:00,0.0,13.6,0.0,975.2,3.9,72.0
2022-12-31 22:00:00,0.0,13.6,0.0,975.2,3.9,72.0
2022-12-31 22:30:00,0.0,13.6,0.0,975.2,3.9,72.0
2022-12-31 23:00:00,0.0,13.6,0.0,975.2,3.9,72.0
2022-12-31 23:30:00,0.0,13.6,0.0,975.2,3.9,72.0
